In [29]:
import numpy as np
import pandas as pd
import glob
import os

In [26]:
# Inspired from https://stackoverflow.com/questions/72885324/find-the-distance-between-letters-in-the-keyboard
# Eucledian norm seems more appropriate than L1 norm

querty = {'q':(0,0), 'w':(1,0), 'e':(2,0), 'r':(3,0), 't':(4,0), 'y':(5,0), 'u':(6,0), 'i':(7,0), 'o':(8,0), 'p':(9,0),
            'a':(0,1), 's':(1,1), 'd':(2,1), 'f':(3,1), 'g':(4,1), 'h':(5,1), 'j':(6,1), 'k':(7,1), 'l':(8,1),
             'z':(0,2), 'x':(1,2), 'c':(2,2), 'v':(3,2), 'b':(4,2), 'n':(5,2), 'm':(5,2),
             'space':(4,3)}

def euclidian_metric(X1: tuple,X2: tuple) -> float:
    """
    Arguments:
        X1 (tuple): key 1 (x,y) coordinate tuple
        X2 (tuple): key 2 (x,y) coordinate tuple

    Returns: Eucledian distance between tuples

    """
    return np.sqrt((X1[0]-X2[0])**2 + (X1[1]-X2[1])**2)

def L1_norm(X1, X2):
    """
    Arguments:
        X1 (tuple): key 1 (x,y) coordinate tuple
        X2 (tuple): key 2 (x,y) coordinate tuple

    Returns: L1 norm between tuples

    """
    return abs(X1[0]-X2[0]) + abs(X1[1]-X2[1])

def Dfcn(l1: str, l2: str) -> float:
    """
    Arguments:
        l1 (str): key 1 str val
        l2 (str): key 2 str val

    Returns: Eucledian distance between keys

    """
    if l1 in querty and l2 in querty and not pd.isna(querty[l1]) and not pd.isna(querty[l2]):
        return euclidian_metric(querty[l1], querty[l2])
    else:
        return np.nan

In [58]:
def safe_mean(arr: pd.Dataframe):
    """
    Arguments:
        arr (pd.Dataframe): input dataframe

    Returns: mean w/o considering inf nor NaN vals

    """
    valid_vals = arr[np.isfinite(arr)]
    if len(valid_vals) > 0:
        return np.mean(valid_vals)
    else:
        return np.nan

In [69]:
directory_path = 'refined2_data/'
output_directory = 'refined2_data_ENG/'

csv_files = glob.glob(f'{directory_path}/*.csv')

averages_df = pd.DataFrame(columns=['pID', 'average_normed_delta_T'])

for file_path in csv_files:
    file_name_csv = os.path.basename(file_path)

    df = pd.read_csv(directory_path+file_name_csv)
    print(df)

    df = df.reset_index(drop=True)

    df.columns = ['key', 'hold_s', 't_release', 't_press']

    df['delta_T'] = -df['t_release'].shift(1) + df['t_press']

    df['dist'] = 0 

    for index in range(1, len(df)):
        df.at[index, 'dist'] = Dfcn(df.iloc[index]['key'], df.iloc[index-1]['key'])

    df['normed_delta_T'] = df['delta_T']/df['dist']
    
    print(df)
    
    file_name = os.path.splitext(file_name_csv)[0]
    
    average_normed_delta_T = safe_mean(df['normed_delta_T'])
    
    averages_df = pd.concat([averages_df, pd.DataFrame({'pID': [re.findall(r'\d+',file_name)[0]], 'average_normed_delta_T': [average_normed_delta_T]})], ignore_index=True)

        
    df.to_csv(output_directory+file_name_csv)

      space  0.1511   8.0693   7.9182
0         q  0.1850  10.6346  10.4495
1         u  0.1344  10.8587  10.7244
2         e  0.1865  11.3900  11.2035
3     space  0.1306  11.7202  11.5896
4         v  0.1712  12.6209  12.4497
...     ...     ...      ...      ...
2183    NaN     NaN      NaN      NaN
2184    NaN     NaN      NaN      NaN
2185    NaN     NaN      NaN      NaN
2186    NaN     NaN      NaN      NaN
2187    NaN     NaN      NaN      NaN

[2188 rows x 4 columns]
        key  hold_s  t_release  t_press  delta_T      dist  normed_delta_T
0         q  0.1850    10.6346  10.4495      NaN  0.000000             NaN
1         u  0.1344    10.8587  10.7244   0.0898  6.000000        0.014967
2         e  0.1865    11.3900  11.2035   0.3448  4.000000        0.086200
3     space  0.1306    11.7202  11.5896   0.1996  3.605551        0.055359
4         v  0.1712    12.6209  12.4497   0.7295  1.414214        0.515834
...     ...     ...        ...      ...      ...       ...           

      key  hold_s  t_release  t_press  delta_T      dist  normed_delta_T
0       t  0.0798     2.5909   2.5111      NaN  0.000000             NaN
1       e  0.0904     2.7927   2.7023   0.1114  2.000000        0.055700
2       n  0.0829     3.1188   3.0359   0.2432  3.605551        0.067452
3       i  0.0770     3.3256   3.2486   0.1298  2.828427        0.045891
4       a  0.0823     3.4865   3.4043   0.0787  7.071068        0.011130
...   ...     ...        ...      ...      ...       ...             ...
2940  NaN     NaN        NaN      NaN      NaN       NaN             NaN
2941  NaN     NaN        NaN      NaN      NaN       NaN             NaN
2942  NaN     NaN        NaN      NaN      NaN       NaN             NaN
2943  NaN     NaN        NaN      NaN      NaN       NaN             NaN
2944  NaN     NaN        NaN      NaN      NaN       NaN             NaN

[2945 rows x 7 columns]
      space  0.2391     2.546    2.3069
0         f  0.1695    2.6591    2.4896
1         u  0.0829

       key  hold_s  t_release    t_press  delta_T      dist  normed_delta_T
0        e  0.2227    17.4130    17.1903      NaN  0.000000             NaN
1        l  0.0710    17.9669    17.8959   0.4829  6.082763        0.079388
2    space  0.0878    18.4796    18.3918   0.4249  4.472136        0.095011
3        v  0.2577    19.7883    19.5306   1.0510  1.414214        0.743169
4        a  0.2931    21.0740    20.7809   0.9926  3.162278        0.313888
..     ...     ...        ...        ...      ...       ...             ...
882      u  0.1006  1045.9400  1045.8394   0.5607  2.828427        0.198237
883      r  0.0782  1046.4233  1046.3451   0.4051  3.000000        0.135033
884      r  0.0704  1046.6805  1046.6101   0.1868  0.000000             inf
885      o  0.0578  1047.3410  1047.2832   0.6027  5.000000        0.120540
886  space  0.1060  1047.8077  1047.7017   0.3607  5.000000        0.072140

[887 rows x 7 columns]
     space  0.0804    0.6026    0.5223
0      NaN  0.1442    3.0

        key  hold_s  t_release   t_press  delta_T      dist  normed_delta_T
0         e  0.1032     2.6989    2.5956      NaN  0.000000             NaN
1         n  0.0502     2.8524    2.8022   0.1033  3.605551        0.028650
2     space  0.0631     3.0648    3.0017   0.1493  1.414214        0.105571
3         t  0.0678     3.3763    3.3085   0.2437  3.000000        0.081233
4         o  0.0921     3.9111    3.8190   0.4427  4.000000        0.110675
...     ...     ...        ...       ...      ...       ...             ...
1419      e  0.1171   724.2580  724.1408   0.0717  5.000000        0.014340
1420      n  0.0723   724.3839  724.3116   0.0536  3.605551        0.014866
1421      t  0.0700   724.7446  724.6746   0.2907  2.236068        0.130005
1422      o  0.0731   725.3378  725.2647   0.5201  4.000000        0.130025
1423      s  0.0927   725.5366  725.4440   0.1062  7.071068        0.015019

[1424 rows x 7 columns]
     space  0.1528  1.7218   1.569
0        c  0.1556  1.8728  

      key  hold_s  t_release   t_press  delta_T      dist  normed_delta_T
0       m  0.1230     3.7194    3.5964      NaN  0.000000             NaN
1       a  0.1969     4.0650    3.8681   0.1487  5.099020        0.029162
2       t  0.0871     4.1909    4.1038   0.0388  4.123106        0.009410
3       i  0.1100     4.5411    4.4312   0.2403  3.000000        0.080100
4       r  0.1230     5.1812    5.0582   0.5171  4.000000        0.129275
...   ...     ...        ...       ...      ...       ...             ...
1404    o  0.1139   734.2707  734.1568  -0.0913  4.123106       -0.022144
1405  NaN  0.1875   734.9395  734.7521   0.4814       NaN             NaN
1406  NaN  0.1742   735.9133  735.7391   0.7996       NaN             NaN
1407  NaN  0.2873   737.7525  737.4652   1.5519       NaN             NaN
1408    e  0.1780   737.8417  737.6637  -0.0888       NaN             NaN

[1409 rows x 7 columns]
     space  0.1631    1.5661    1.4031
0      NaN  0.1503    6.7996    6.6493
1        

...

      key  hold_s  t_release  t_press  delta_T      dist  normed_delta_T
0       r  0.1592     3.8588   3.6996      NaN  0.000000             NaN
1       e  0.1348     4.2172   4.0823   0.2235  1.000000        0.223500
2       c  0.3437     5.3771   5.0335   0.8163  2.000000        0.408150
3       o  0.2032     5.7214   5.5182   0.1411  6.324555        0.022310
4       g  0.2647     6.8476   6.5830   0.8616  4.123106        0.208969
...   ...     ...        ...      ...      ...       ...             ...
1639  NaN     NaN        NaN      NaN      NaN       NaN             NaN
1640  NaN     NaN        NaN      NaN      NaN       NaN             NaN
1641  NaN     NaN        NaN      NaN      NaN       NaN             NaN
1642  NaN     NaN        NaN      NaN      NaN       NaN             NaN
1643  NaN     NaN        NaN      NaN      NaN       NaN             NaN

[1644 rows x 7 columns]
     space  0.1078   5.317  5.2092
0        e  0.1660  6.0589  5.8930
1        n  0.1361  6.4886  6

     key  hold_s  t_release   t_press  delta_T      dist  normed_delta_T
0      m  0.1043     3.0665    2.9621      NaN  0.000000             NaN
1      e  0.1068     3.1510    3.0442  -0.0223  3.605551       -0.006185
2      n  0.1326     3.2888    3.1562   0.0052  3.605551        0.001442
3      t  0.0841     3.3555    3.2714  -0.0174  2.236068       -0.007782
4      i  0.1422     3.5283    3.3861   0.0306  3.000000        0.010200
...   ..     ...        ...       ...      ...       ...             ...
3417   n  0.0885   718.9093  718.8208   0.0927  3.605551        0.025710
3418   a  0.1244   718.9798  718.8554  -0.0539  5.099020       -0.010571
3419   s  0.0924   719.1285  719.0360   0.0562  1.000000        0.056200
3420   t  0.0809   719.3414  719.2605   0.1320  3.162278        0.041742
3421   e  0.1448   719.4929  719.3482   0.0068  2.000000        0.003400

[3422 rows x 7 columns]
      space  0.0846  7.5556  7.4711
0         y  0.0969  8.7041  8.6072
1         o  0.0785  8.8985

        key  hold_s  t_release  t_press  delta_T      dist  normed_delta_T
0       NaN  0.0309     1.6728   1.6419      NaN  0.000000             NaN
1         q  0.1366     1.7712   1.6346  -0.0382       NaN             NaN
2         u  0.0548     1.8603   1.8054   0.0342  6.000000        0.005700
3         e  0.0667     2.0250   1.9583   0.0980  4.000000        0.024500
4     space  0.1109     2.1948   2.0839   0.0589  3.605551        0.016336
...     ...     ...        ...      ...      ...       ...             ...
4990    NaN     NaN        NaN      NaN      NaN       NaN             NaN
4991    NaN     NaN        NaN      NaN      NaN       NaN             NaN
4992    NaN     NaN        NaN      NaN      NaN       NaN             NaN
4993    NaN     NaN        NaN      NaN      NaN       NaN             NaN
4994    NaN     NaN        NaN      NaN      NaN       NaN             NaN

[4995 rows x 7 columns]
      space  0.1204    8.6894     8.569
0         m  0.1073    9.3386    9.

      key  hold_s  t_release  t_press  delta_T      dist  normed_delta_T
0       m  0.0953     2.9560   2.8607      NaN  0.000000             NaN
1       a  0.0985     3.1653   3.0668   0.1108  5.099020         0.02173
2       e  0.0916     3.8475   3.7558   0.5905  2.236068         0.26408
3       s  0.1421     4.1782   4.0360   0.1885  1.414214         0.13329
4       t  0.1110     4.5850   4.4740   0.2958  3.162278         0.09354
...   ...     ...        ...      ...      ...       ...             ...
2404  NaN     NaN        NaN      NaN      NaN       NaN             NaN
2405  NaN     NaN        NaN      NaN      NaN       NaN             NaN
2406  NaN     NaN        NaN      NaN      NaN       NaN             NaN
2407  NaN     NaN        NaN      NaN      NaN       NaN             NaN
2408  NaN     NaN        NaN      NaN      NaN       NaN             NaN

[2409 rows x 7 columns]
      space  0.1727  2.9198  2.7471
0         e  0.0816  4.1802  4.0985
1         r  0.0905  4.3882

        key  hold_s  t_release  t_press  delta_T      dist  normed_delta_T
0         y  0.0639     2.4938   2.4299      NaN  0.000000             NaN
1     space  0.0803     2.7856   2.7053   0.2115  3.162278        0.066882
2         l  0.0634     3.2833   3.2199   0.4343  4.472136        0.097112
3         a  0.2834     3.3262   3.0428  -0.2405  8.000000       -0.030062
4     space  0.0872     4.0526   3.9654   0.6392  4.472136        0.142929
...     ...     ...        ...      ...      ...       ...             ...
4665    NaN     NaN        NaN      NaN      NaN       NaN             NaN
4666    NaN     NaN        NaN      NaN      NaN       NaN             NaN
4667    NaN     NaN        NaN      NaN      NaN       NaN             NaN
4668    NaN     NaN        NaN      NaN      NaN       NaN             NaN
4669    NaN     NaN        NaN      NaN      NaN       NaN             NaN

[4670 rows x 7 columns]
     space  0.3552  5.5123  5.1571
0        n  0.1194  6.3492  6.2298
1    

In [72]:
averages_df.to_csv('feature_eng.csv')

     pID  average_normed_delta_T
0     68                0.227325
1     97               -0.185779
2     82                0.579151
3     85                0.136303
4   1029                0.117904
..   ...                     ...
80    92                0.194707
81    78                0.144300
82    87                0.065799
83    80                0.208118
84  1039                0.044175

[85 rows x 2 columns]
